# **Introduction:**

This file serves to compare the inference systems that had been developed.

**Date Created: 16/2/2025**

**Date Modified: 16/2/2025**

# **Import Packages:**

This section imports the necessary packages.

In [43]:
# import packages:
import numpy as np
import time
from keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from sklearn.preprocessing import StandardScaler
from pickle import load
from ANFIS_Custom_Layers import *
from PythonFISFunctionV3 import *


# **Create FIS:**

This section creates the FIS.

In [44]:
rulebase = fis_create()

# **Load the ANFIS Model:**

This section loads the ANFIS model and the scaler.

In [45]:
# define the dictionary of custom objects:
custom_objects = {
    # # layers:
    'MF_Layer'          : MF_Layer,
    'FS_Layer'          : FS_Layer,
    'NM_Layer'          : NM_Layer,
    'CN_Layer'          : CN_Layer,
    'O_Layer'           : O_Layer,

    # other:
    'OrderedConstraint' : OrderedConstraint(),
    'mse'               : MeanSquaredError()
}

# load the model:
model = load_model('anfis_model.h5', custom_objects = custom_objects)

# load the scaler:
scaler = load(open('scaler.pkl', 'rb'))

# **Test the Models:**

This section tests the models against one another on the same input dataset

In [46]:
# define the input values:
input_data = np.array([[2, 14.024, 34.047], 
                      [0, 35.213, 0], 
                      [1, 15.374, 18.783], 
                      [1, 14.620, 11.881]])

# scale the input using the scaler:
scaled_input = scaler.transform(input_data)

Test the FIS:

In [47]:
# get the start time:
fis_start_time = time.time()
suitability = []

# suitability calculation:
for a,b,c in input_data:
    suit = fis_solve(rulebase, a, b, c)
    suitability.append(suit)

# final time:
fis_time = time.time() - fis_start_time
print(f'inference time of the FIS: {round(fis_time, 3)}')
print(suitability)


inference time of the FIS: 0.315
[np.float64(4.112042796178686), np.float64(5.0), np.float64(5.5333160818597635), np.float64(6.322235972052975)]


Test the ANFIS:

In [49]:
# get the start time:
anfis_start_time = time.time()

# suitability calculation:
prediction = model.predict(scaled_input)

# final time:
anfis_time = time.time() - anfis_start_time
print(f'inference time of the ANFIS: {round(anfis_time, 3)}')
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
inference time of the ANFIS: 0.032
[[4.3295665]
 [4.7907586]
 [5.3551536]
 [5.9129276]]
